In [2]:
TF_ENABLE_ONEDNN_OPTS=1
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from pathlib import Path
import scipy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import skimage
import glob
import matplotlib.image as mpimg
import imghdr
import numpy.ma as ma
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models

In [1]:
pip install scikit-image

  Using cached scikit_image-0.19.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
  Using cached scipy-1.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached imageio-2.25.1-py3-none-any.whl (3.4 MB)
  Using cached networkx-3.0-py3-none-any.whl (2.0 MB)
  Using cached PyWavelets-1.4.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.9 MB)
  Using cached tifffile-2023.2.3-py3-none-any.whl (215 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

  Using cached scikit_learn-1.2.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Print GPUS

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-02-14 22:54:14.414005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 22:54:14.421581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 22:54:14.422216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
## Image generator
image_generator = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2],# brightness
        validation_split=0.2,)

In [8]:
## Train and Val datasets

train_dataset = image_generator.flow_from_directory('Train_Set/',
                                          target_size = (224, 224),
                                          shuffle=True,
                                          subset="training",
                                          batch_size = 32,
                                          class_mode = 'binary')

val_dataset = image_generator.flow_from_directory('Validation_Set/',
                                          target_size = (224, 224),
                                          shuffle=True,
                                          subset="validation",
                                          batch_size = 32,
                                          class_mode = 'binary')

Found 421 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [9]:
## Modelo base
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [224, 224,3]),
    keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.Conv2D(128, (2, 2), activation='relu'),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(2, activation ='relu')
])
model.compile(optimizer='sgd',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

2023-02-14 22:57:23.933931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 22:57:23.934435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 22:57:23.935228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 22:57:23.935840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

In [10]:
## Entrenar modelo
model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20


2023-02-14 22:57:30.207385: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-14 22:57:30.876328: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-14 22:57:30.877257: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-14 22:57:30.877296: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-14 22:57:30.878343: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-14 22:57:30.878434: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


14/14 [==============================] - 16s 669ms/step - loss: 4.8827 - accuracy: 0.3254 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 2/20
14/14 [==============================] - 8s 522ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 3/20
14/14 [==============================] - 8s 522ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 4/20
14/14 [==============================] - 8s 523ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 5/20
14/14 [==============================] - 8s 525ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 6/20
14/14 [==============================] - 8s 533ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 7/20
14/14 [==============================] - 8s 525ms/step - loss: 4.8899 - accuracy: 0.3207 - val_loss: 2.9755 - val_accuracy: 0.1951
Epoch 8/20
14/14